# Извлечение именованных сущностей.

In [34]:
from google.colab import drive
drive.mount('/content/gdrive') #, force_remount=True

Mounted at /content/gdrive


In [35]:
import os
os.chdir('gdrive/My Drive/Colab Notebooks/nlp_hw')

In [ ]:
!pip install natasha ipymarkup

In [2]:
!pip install pymorphy2

In [3]:
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

In [4]:
p = morph.parse('Михаил')[0].tag
print('Тэги - ', p)
print('Name' in p) #тэг имени

Тэги -  NOUN,anim,masc,Name sing,nomn
True


In [5]:
p = morph.parse('Иванов')[0].tag
print('Тэги - ', p)
print('Surn' in p) #тэг фамилии

Тэги -  NOUN,anim,masc,Sgtm,Surn sing,nomn
True


In [6]:
p = morph.parse('Петрович')[0].tag
print('Тэги - ', p)
print('Patr' in p) #тэг отчества

Тэги -  NOUN,anim,masc,Patr sing,nomn
True


In [7]:
p = morph.parse('Одинцово')[0].tag
print('Тэги - ', p)
print('Geox' in p) #тэг локация

Тэги -  NOUN,inan,neut,Sgtm,Geox sing,nomn
True


In [8]:
p = morph.parse('Яндекс')[0].tag
print('Тэги - ', p)
print('Orgn' in p) #тэг организация

Тэги -  NOUN,inan,masc,Orgn sing,nomn
True


In [9]:
p = morph.parse('МГУ')[0].tag
print('Тэги - ', p)
print('Orgn' in p) #тэг организация

Тэги -  NOUN,inan,masc,Sgtm,Fixd,Abbr,Orgn sing,gent
True


In [10]:
p = morph.parse('ВШЭ')[0].tag
print('Тэги - ', p)
print('Orgn' in p) #тэг организация

Тэги -  NOUN,inan,masc,Sgtm,Fixd,Abbr,Orgn sing,nomn
True


Попробуем разметить дургих слова тех же сущностей

In [11]:
p = morph.parse('МГУ')[0].tag
print('Тэги - ', p)
print('Orgn' in p) #тэг организация

Тэги -  NOUN,inan,masc,Sgtm,Fixd,Abbr,Orgn sing,gent
True


In [12]:
p = morph.parse('НИУ ВШЭ')[0].tag
print('Тэги - ', p)
print('Orgn' in p) #тэг организация

Тэги -  NOUN,inan,masc,Sgtm,Fixd,Abbr,Orgn sing,nomn
True


In [13]:
p = morph.parse('Гугл')[0].tag
print('Тэги - ', p)
print('Orgn' in p) #тэг организация

Тэги -  NOUN,inan,masc,Sgtm,Orgn sing,nomn
True


In [15]:
p = morph.parse('Аби')[0].tag
print('Тэги - ', p)
print('Orgn' in p) #тэг организация

p = morph.parse('Абби')[0].tag
print('Тэги - ', p)
print('Orgn' in p) #тэг организация

Тэги -  NOUN,anim,masc,Sgtm,Fixd,Name sing,nomn
False
Тэги -  NOUN,inan,neut,Fixd sing,nomn
False


In [16]:
p = morph.parse('Чертаново')[0].tag
print('Тэги - ', p)
print('Geox' in p) #тэг локация

Тэги -  NOUN,inan,neut,Sgtm,Geox sing,nomn
True


In [17]:
p = morph.parse('Иван Иванов')[0].tag
print('Тэги - ', p)
print('Name' in p) #тэг имени
print('Surn' in p) #тэг фамилии
print('Patr' in p) #тэг отчества

Тэги -  NOUN,anim,masc,Name plur,gent
True
False
False


In [20]:
p = morph.parse('Семирамида')[0].tag
print('Тэги - ', p)
print('Name' in p) #тэг имени

Тэги -  NOUN,anim,femn,Name sing,nomn
True


In [21]:
p = morph.parse('Муза')[0].tag
print('Тэги - ', p)
print('Name' in p) #тэг имени

Тэги -  NOUN,anim,femn sing,nomn
False


Другой инструмент - natasha (https://github.com/natasha/natasha)

До недавнего времени это была просто библиотека с правилами, написанными на yargy https://github.com/natasha/yargy (специальной грамматике на питоне), а теперь это уже многоцелевая русскоязычная нлп-библиотека. Извлекать сущности тут можно либо предобученной моделью либо правилами.

In [22]:
from natasha import (
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    PER,
    NamesExtractor,
    Doc,
    DatesExtractor,
    MoneyExtractor,
    AddrExtractor,
    
)


# для красивой отрисовки есть отдельная библиотека
from ipymarkup import show_span_box_markup


# интерфейс загрузки моделей пока не самый простой
# со временем наверное станет получше
segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
ner_tagger = NewsNERTagger(emb)


# правила загружаются вот так
names_extractor = NamesExtractor(morph_vocab)
dates_extractor = DatesExtractor(morph_vocab)
money_extractor = MoneyExtractor(morph_vocab)
addr_extractor = AddrExtractor(morph_vocab)

In [23]:
text = 'Влад Веселов. Андрей Петрович. Алиса Викторовна. Студия Артемия Лебедева'
# для моделей нужно превратить текст в класс Doc (похоже на spacy)
doc = Doc(text)


# потом модели применяются к тексту
doc.segment(segmenter) # сегментер нужно применить, иначе нер не будет работать
doc.tag_ner(ner_tagger)
show_span_box_markup(text, doc.spans)

In [24]:
matches = names_extractor(text)
spans = [(_.start, _.stop, 'PER') for _ in matches]
show_span_box_markup(text, spans)

In [25]:
text = 'Более того в Москве, на улице Такой. На реке Оке. В германии'

doc = Doc(text)
doc.segment(segmenter)
doc.tag_ner(ner_tagger)
show_span_box_markup(text, doc.spans)

In [26]:
matches = addr_extractor(text)
spans = [(_.start, _.stop, 'LOC') for _ in matches]
show_span_box_markup(text, spans)

In [27]:
text = 'ФСБ. Московский государственный университет. Высшая школа экономики. ВШЭ. Mail.ru '
doc = Doc(text)
doc.segment(segmenter)
doc.tag_ner(ner_tagger)
show_span_box_markup(text, doc.spans)

In [28]:
text = 'С 2015 г. по 2017 год. 16 апреля 1993 года. В четверг . 23.12.18'

matches = dates_extractor(text)
spans = [(_.start, _.stop) for _ in matches]
show_span_box_markup(text, spans)

In [29]:
text = "Он заплатил ему 3,000,000,000 рублей."

matches = money_extractor(text)
spans = [(_.start, _.stop) for _ in matches]
show_span_box_markup(text, spans)

In [30]:
text = "Он заплатил ему 3000 000 000 рублей."

matches = money_extractor(text)
spans = [(_.start, _.stop) for _ in matches]
show_span_box_markup(text, spans)

Если нужно улучшить выделение стандартныйх типов или научиться извлекать специфичные сущности, нужно писать правила в Yargy. 

In [42]:
from yargy import Parser, rule, or_
from yargy.predicates import in_, in_caseless
from yargy.tokenizer import MorphTokenizer
from yargy.pipelines import morph_pipeline, caseless_pipeline
from yargy.interpretation import fact
from IPython.display import display

Загрузим тексты

In [36]:
with open('avito_cars.txt', 'r', encoding='utf-8') as f:
  data = f.readlines()

Сделаем правило для марок.


In [56]:
Auto = fact(
    'Auto',
    ['name', 'model']
)

In [92]:
Vaz = rule(
    morph_pipeline(['Ваз', 'ваз', 'ВАЗ']).interpretation(Auto.name),
    morph_pipeline(['21110', '21063', '21074']).interpretation(Auto.model))

In [93]:
Volvo = rule(
    morph_pipeline(['Volvo', 'volvo', 'VOLVO', 'Вольво', 'вольво', 'ВОЛЬВО']).interpretation(Auto.name),
    morph_pipeline(['xc60', 'XC60']).interpretation(Auto.model)    
    )

In [94]:
AUTO = or_(Vaz, Volvo).interpretation(Auto)

parser = Parser(AUTO)

Попробуем сматчить по текстам.

In [96]:
matches = []

for sent in data:
    for match in parser.findall(sent):
        matches.append(match.fact)

In [97]:
len(matches)

97

In [98]:
for m in matches[:10]:
    print(m.name, m.model)

ВАЗ 21063
ВАЗ 21074
ВАЗ 21074
ваз 21063
ваз 21063
ВАЗ 21074
ВАЗ 21074
ВАЗ 21074
Ваз 21074
ВАЗ 21074


Добавим опциональные параметры и нормализацию

In [171]:
Auto = fact(
    'Auto',
    ['name', 'model', 'props']
)

VERSIONS = {
    '1.8': '1.8 л.',
    '125': '125 л.с.',
    '(125л.с.)': '125 л.с.',
    '1.6': '1.6 л.',
    '1,5' : '1.5 л.',
    'мотор 1.6': 'мотор 1.6',
    'пробег 118т.км.': 'пробег 118 т.км.',
    'дв. 8 кл': 'дв. 8 кл.',
    'LADA SAMARA' : 'LADA SAMARA',
    '2008' : '2008 г. выпуска',
    '2006' : '2006 г. выпуска',
    '2003' : '2003 г. выпуска'
}

Vaz = rule(
    morph_pipeline(['Ваз', 'ваз', 'ВАЗ']).interpretation(Auto.name.const('ВАЗ')),
    morph_pipeline(['21110', '21063', '21074', '211440','2114', '2110', '2109', '2107']).interpretation(Auto.model.normalized()),
    caseless_pipeline(VERSIONS).interpretation(Auto.props.normalized().custom(VERSIONS.get)).optional())

Volvo = rule(
    morph_pipeline(['Volvo', 'volvo', 'VOLVO', 'Вольво', 'вольво', 'ВОЛЬВО']).interpretation(Auto.name.const('Volvo')),
    morph_pipeline(['xc60', 'XC60', 'ХС70', 'S40', 'S', 'C']).interpretation(Auto.model.normalized()),   
    #in_('xc60XC60ХС70S40').interpretation(Auto.model.normalized()),
    caseless_pipeline(VERSIONS).interpretation(Auto.props.normalized().custom(VERSIONS.get)).optional())

AUTO = or_(Vaz, Volvo).interpretation(Auto)

parser = Parser(AUTO)

In [176]:
list(parser.findall('ваз 21110 дв. 8 кл'))[0].fact

Auto(
    name='ВАЗ',
    model='21110',
    props='дв. 8 кл.'
)

In [172]:
matches = []

for sent in data:
    for match in parser.findall(sent):
        matches.append(match.fact)

In [173]:
len(matches)

1472

In [175]:
for m in matches[170:200]:
  print(m.name, m.model, m.props)

ВАЗ 2107 None
ВАЗ 2107 None
ВАЗ 2107 None
ВАЗ 2114 None
ВАЗ 2109 None
ВАЗ 2114 None
ВАЗ 2107 None
ВАЗ 2110 None
ВАЗ 2109 None
ВАЗ 2114 None
ВАЗ 2110 None
ВАЗ 2107 None
ВАЗ 2114 2006 г. выпуска
ВАЗ 2114 2008 г. выпуска
ВАЗ 2110 None
ВАЗ 2110 None
ВАЗ 2110 None
ВАЗ 2114 None
ВАЗ 2109 None
ВАЗ 2109 None
ВАЗ 2107 None
ВАЗ 21074 None
ВАЗ 2109 None
ВАЗ 2114 2003 г. выпуска
Volvo S40 1.8 л.
Volvo S40 1.8 л.
ВАЗ 2114 None
ВАЗ 2107 None
ВАЗ 2110 None
ВАЗ 2107 None
